In [1]:
import numpy as np
import pandas as pd

## Loads

In [2]:
L=7.5*10**3 #mm
combo1=pd.Series({"Ned":300, "Hed":30}, name="Combo1")
combo2=pd.Series({"Ned":500, "Hed":10}, name="Combo2")
combo3=pd.Series({"Ned":250, "Hed":40}, name="Combo3")

loads=pd.DataFrame([combo1, combo2, combo3])
loads["Med"]=loads.Hed*L*10**(-3)
loads

,Ned,Hed,Med
Combo1,300,30,225.0
Combo2,500,10,75.0
Combo3,250,40,300.0


## Materials

In [3]:
fyk=355 #N/mm^2
E=200000 #Mpa
G = 81000 #N/mm^2

## Cross-section

In [4]:
# HE300B
h=300 #mm (section height)
b=300 #mm (section width)
tw=11 #mm (web width)
tf=19 #mm (flange width)
A=161.3*10**2 #mm^2 (total area)
Iy=25170*10**4 #mm^4 (second moment of area, y axis)
iy=13*10 #mm (radius of gyration, y axis)
Wy=1678*10**3 #mm^3 (elastic section modulus, y axis)

Iz=8563*10**4 #mm^4 (second moment of area, z axis)
iz=7.58*10 #mm (radius of gyration, z axis)
Wz=570.9*10**3 #mm^3 (elastic section modulus, z axis)

It=185*10**4 #mm^4 (torsional constant)
Iw=(Iz*(h-tf)**2)/4 #mm^6 (warping constant)

## Buckling

In [5]:
#buckling curve:
print(h/b)

1.0


In [6]:
epsilon = np.sqrt(355/fyk)
lambda_1 = np.pi*np.sqrt(E/fyk)
print("epsilon= %.2f"%(epsilon))
print("lambda_1= %.2f"%(lambda_1))

epsilon= 1.00
lambda_1= 74.57


In [7]:
alpha_y = 0.21
Lcr_y = 2*L
lambda_y = (Lcr_y/iy)*(1/lambda_1)
phi_y = 0.5*(1+alpha_y*(lambda_y-0.2)+lambda_y**2)
chi_y = 1/(phi_y+np.sqrt(phi_y**2-lambda_y**2))
print("alpha_y= %.2f"%(alpha_y))
print("Lcr_y= %.2f"%(Lcr_y))
print("lambda_y= %.2f"%(lambda_y))
print("phi_y= %.2f"%(phi_y))
print("chi_y= %.7f"%(chi_y))

alpha_y= 0.21
Lcr_y= 15000.00
lambda_y= 1.55
phi_y= 1.84
chi_y= 0.3531319


In [8]:
k=0.7
kw=0.7
C1 = 2.092
alpha_lt = 0.34
lambda_lt_0 = 0.4
beta = 0.75

Mcr = C1*np.pi**2*E*Iz/((k*L)**2)*np.sqrt((k/kw)*Iw/(Iz)\
                    +(k*L)**2*G*It/(np.pi**2*E*Iz))*1e-6

lambda_lt = np.sqrt(Wy*fyk/(Mcr*1e6))
phi_lt = 0.5*(1+alpha_lt*(lambda_lt-lambda_lt_0)+beta*lambda_lt**2)
chi_lt = 1/(phi_lt+np.sqrt(phi_lt**2-beta*lambda_lt**2))

print("Mcr= %.2f"%(Mcr))
print("lambda_lt= %.2f"%(lambda_lt))
print("phi_lt= %.2f"%(phi_lt))
print("chi_lt= %.2f"%(chi_lt))

Mcr= 2696.44
lambda_lt= 0.47
phi_lt= 0.59
chi_lt= 0.97


In [9]:
Cmy = 0.6
Cmz = 0.6

kyy = Cmy*1.8
kzz = Cmz*2.4
kyz = 0.6*kzz
kzy=1

print("kyy= %.2f"%(kyy))
print("kzz= %.2f"%(kzz))
print("kyz= %.2f"%(kyz))
print("kzy= %.2f"%(kzy))

kyy= 1.08
kzz= 1.44
kyz= 0.86
kzy= 1.00


In [10]:
Nrk = A*fyk*0.001   # kN
Mrk_y = Wy*fyk*1e-6  # kNm

print("Nrk= %.2f"%(Nrk))
print("Mrk_y= %.2f"%(Mrk_y))

Nrk= 5726.15
Mrk_y= 595.69


In [11]:
gamma_m1 = 1.05
results=pd.DataFrame()
results["check1"] = loads.Ned/(chi_y*Nrk/gamma_m1)\
            +kyy*(loads.Med)/(chi_lt*Mrk_y/gamma_m1)
results["check2"] = loads.Ned/(chi_y*Nrk/gamma_m1)\
            +kzy*(loads.Med)/(chi_lt*Mrk_y/gamma_m1)
results

,check1,check2
Combo1,0.596262,0.563633
Combo2,0.406460,0.395584
Combo3,0.717126,0.673621
